# Lokaverkefni: Leiðarval hjá Strætó

In [ ]:
from posixpath import split

def convertTime(t1, t2):
  t1_hours = int(t1.split(':')[0])
  t1_min = int(t1.split(':')[1])
  t1_seconds = int(t1.split(':')[2])

  t2_hours = int(t2.split(':')[0])
  t2_min = int(t2.split(':')[1])
  t2_seconds = int(t2.split(':')[2])


  t1_minutes = (t1_hours * 60 ) + t1_min + (t1_seconds / 60)
  t2_minutes = (t2_hours * 60 ) + t2_min + (t2_seconds / 60)

  diff = round((max(t1_minutes, t2_minutes) - min(t1_minutes, t2_minutes)), 2)

  return diff


t1 = '09:30:20'
t2 = '09:10:00'
print(convertTime(t1, t2))

In [ ]:
def time_to_minutes(ti):
  splitted = ti.split(':')
  
  hours = int(splitted[0])
  min = int(splitted[1])
  seconds = int(splitted[2])

  return (hours * 60) + min + (seconds / 60)

In [ ]:
# Graph stuff

def add_vertex(g, v):
  if v in g:
    raise Exception("Hnúturinn er þegar í netinu")
  else:
    g[v] = []
  return g

def add_edge(g, fro, to, data):
  if fro not in g:
    raise Exception(fro + " er ekki í netinu")
  if to not in g:
    raise Exception(to + " er ekki í netinu")
  g[fro].append([data, to])
  return g

def get_edges(g, v):
  return g[v]

In [ ]:
# Stop information
#0: stop_id
#1: stop_name
#2: stop_lat
#3: stop_lon
#4: location_type
stoppistodvar = []
with open(sys.path[0]+'/gtfs/stops.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None)  # skip the header
    for row in spamreader:
        stoppistodvar.append(row)

stoppistodvar_id_to_name = {x[0]: x[1] for x in stoppistodvar}


# Weekdays only
weekday_service_ids = set()
with open(sys.path[0]+'/gtfs/calendar_dates.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None)  # skip the header
    for row in spamreader:
        service_id, date, exception_type = row
        if 'MTWTF' in service_id and exception_type == '1':
            weekday_service_ids.add(service_id)

#0: route_id
#1: service_id
#2: trip_id
#3: trip_headsign
#4: trip_short_name
#5: direction_id
#6: block_id
#7: shape_id
trip_to_straeto = {}
with open(sys.path[0]+'/gtfs/trips.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None)  # skip the header
    for row in spamreader:
        if row[0][:2] == ('ST') and (row[0] not in ['ST.101','ST.102','ST.103','ST.104','ST.105','ST.106']) and (row[1] in weekday_service_ids):
            trip_to_straeto[row[2]] = row[0]

#0: trip_id
#1: arrival_time
#2: departure_time
#3: stop_id
#4: stop_sequence
#5: stop_headsign
#6: pickup_type
leidir = {}
with open(sys.path[0]+'/gtfs/stop_times.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None)  # skip the header
    for row in spamreader:
        if row[6] == '0' and row[0] in trip_to_straeto:
            if row[0] not in leidir:
                leidir[row[0]] = []
            leidir[row[0]].append(row)


In [ ]:
# Making the graph
graph = {}
vertex_info = {}

for x in stoppistodvar:
    graph = add_vertex(graph, x[0])
    vertex_info[x[0]] = [x[0], stoppistodvar_id_to_name[x[0]], 'stöð']

for path in paths:
    for s in range(len(paths[path])-1):
        interval_name = 'p' + path + '_' + paths[path][s][4]
        graph = add_vertex(graph, interval_name)
        vertex_info[interval_name] = [path, trip_to_straeto[path], 'strætó']
        graph = add_edge(graph, paths[path][s][3], interval_name, paths[path][s][2])
        graph = add_edge(graph, interval_name, paths[path][s+1][3], paths[path][s+1][1])


print("Number of nodes:", len(graph))
print("Number of edges:", sum(len(graph[x]) for x in graph))

In [ ]:
import math
import sys

stoppistodvar = []
with open(sys.path[0]+'/gtfs/stops.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None) 
    for row in spamreader:
        stoppistodvar.append(row)

stop_locations = {}
for row in stoppistodvar:
    stop_id = row[0]
    stop_lat = float(row[2])
    stop_lon = float(row[3])
    stop_locations[stop_id] = (stop_lat, stop_lon)

for path in paths:
    for s in range(len(paths[path])-1):
        interval_name = 'p' + path + '_' + paths[path][s][4]
        stop_locations[interval_name] = stop_locations[paths[path][s+1][3]]

lat, lon = stop_locations['10000802']
# print("Latitude:", lat, "Longitude:", lon)

def calculate_distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    R = 6371000
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d

# print('Distance in meters', calculate_distance(stop_locations['20000250'], stop_locations['20000251']))

MAX_WALKING_DISTANCE = 300  

def add_walking_edges(graph, stop_locations):
    stop_ids = list(stop_locations.keys())
    for i, stop_id1 in enumerate(stop_ids):
        if stop_id1[0] == 'p':
            continue
        for stop_id2 in stop_ids[i+1:]:  
            if stop_id2[0] == 'p':
                continue
            distance = calculate_distance(stop_locations[stop_id1], stop_locations[stop_id2])
            if distance <= MAX_WALKING_DISTANCE:
                try:
                    graph = add_edge(graph, stop_id1, stop_id2, distance)
                    graph = add_edge(graph, stop_id2, stop_id1, distance)
                except Exception as e:
                    print('Exception', e) 
    return graph

# add_walking_edges(graph, stop_locations)

def print_walkable_stops_from(graph, stop_id, stop_locations):
    if stop_id not in stop_locations:
        print("Stop ID not found in stop locations.")
        return
    
    origin = stop_locations[stop_id]
    print(f"Walkable locations from stop {stop_id}:")
    
    for other_stop_id, destination in stop_locations.items():
        if stop_id == other_stop_id:  
            continue
        
        distance = calculate_distance(origin, destination)
        if distance <= MAX_WALKING_DISTANCE:
            print(f"- To stop {other_stop_id}: {distance:.2f} meters")

print_walkable_stops_from(graph, '10000814', stop_locations)


In [ ]:
# Making the graph with walking edges

walk_graph = {}

for x in stoppistodvar:
  walk_graph = add_vertex(walk_graph, x[0])
walk_graph = add_walking_edges(walk_graph, stop_locations)

print(len(walk_graph))

for x in walk_graph['20000254']:
  print(x)

In [ ]:
def minutes_to_time(t):
    t = int(t)
    hours = t // 60
    minutes = t % 60
    return f'{hours:02d}:{minutes:02d}'

print(minutes_to_time(760))

In [ ]:
# g: graph
# g_walk: graph of walkable
# x: origin
# y: destination
# t: beginning time

def find_path(g, g_walk, x, y, t):
    t = time_to_minutes(t)
    if x not in g:
        raise Exception(x + " er ekki í netinu")
    unvisited_nodes = list(g.keys())
    earliest = {}
    previous_nodes = {}
    for node in unvisited_nodes:
        earliest[node] = sys.maxsize
    earliest[x] = t
    while unvisited_nodes: # Need to find an algorithm that isn't O(n^2)
        # Change to use a priority queue instead of searching through the entire list of unvisited nodes?
        current_min_node = None
        for node in unvisited_nodes:
            if current_min_node == None:
                current_min_node = node
            elif earliest[node] < earliest[current_min_node]:
                current_min_node = node
        if earliest[current_min_node] == sys.maxsize:
            break
        e = get_edges(g, current_min_node)
        if current_min_node == y:
            break
        for edge in e:
            if earliest[edge[1]] >= time_to_minutes(edge[0]) and time_to_minutes(edge[0]) >= earliest[current_min_node]:
                earliest[edge[1]] = time_to_minutes(edge[0])
                previous_nodes[edge[1]] = current_min_node
        if current_min_node in g_walk:
            e = get_edges(g_walk, current_min_node)
            for edge in e:
                time_to_reach = earliest[current_min_node] + edge[0]/40 # 40 meters per minute
                if earliest[edge[1]] >= time_to_reach:
                    earliest[edge[1]] = time_to_reach
                    previous_nodes[edge[1]] = current_min_node
        unvisited_nodes.remove(current_min_node)
    output = []
    bus_trip = y
    output.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    while bus_trip != x:
        bus_trip = previous_nodes[bus_trip]
        output.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    return output[::-1]

meistaravellir_to_fb = find_path(graph, walk_graph, '90000097', '90000446', "11:30:00")
# meistaravellir_to_fb = find_path(graph, {}, '90000097', '90000446', "11:30:00")

In [ ]:
import time

def time_find_path(graph, walk_graph, test_cases):
    elapsed_times = [] 
    for x, y, t in test_cases:
        start_time = time.time()
        find_path(graph, walk_graph, x, y, t)
        end_time = time.time()
        elapsed = end_time - start_time  
        elapsed_times.append(elapsed)
    
    average_time = sum(elapsed_times) / len(elapsed_times)
    requests_per_second = 1 / average_time if average_time > 0 else float('inf')
    print(f"Average execution time: {average_time:.4f} seconds")
    print(f"Requests per second: {requests_per_second:.2f}")


test_cases = [
    ('90000097', '90000446', "11:30:00"), # meistaravellir to fb
    ('10000802', '10000804', "12:30:00"), # hamraborg to sunnuhlíð
    ('90000410', '90000296', "08:00:00"), # skógarás to háskóli íslands
    ('90000780', '10000875', "15:50:00"), # borgartún to smáralind
    ('14001641', '90000173', "17:30:00"), # flatahraun to dunhagi
    ('13001337', '13001300', "18:09:00"), # Asparlundur to Ásgarður
    ('16040620', '90000213', "20:30:00"), # Fmos to laugardalslaug
    ('90000351', '90000461', "11:43:00"), # ártún to kringlan
    ('90000077', '90000301', "09:15:00"), # bústaðakirkja to perlan
    ('90000446', '90000097', "11:30:00"), # fb to meistaravellir
    ]

time_find_path(graph, walk_graph, test_cases)


In [ ]:
import heapq

def find_path_new(g, g_walk, x, y, t):
    t = time_to_minutes(t)
    if x not in g:
        raise Exception(x + " er ekki í netinu")

    pq = []
    heapq.heappush(pq, (t, x))

    earliest = {node: float('inf') for node in g}
    earliest[x] = t

    previous_nodes = {}


    while pq:
        current_distance, current_node = heapq.heappop(pq)

        if current_node == y:
            break

        if current_distance > earliest[current_node]:
            continue

        
        for edge in get_edges(g, current_node):
            next_stop, travel_time = edge[1], time_to_minutes(edge[0])
            new_distance = travel_time
            if new_distance < earliest[next_stop] and travel_time >= current_distance:
                earliest[next_stop] = new_distance
                previous_nodes[next_stop] = current_node
                heapq.heappush(pq, (new_distance, next_stop))

        if current_node in g_walk:
            for edge in get_edges(g_walk, current_node):
                next_stop, walking_time = edge[1], edge[0] / 40  
                new_distance = current_distance + walking_time
                if new_distance < earliest[next_stop]:
                    earliest[next_stop] = new_distance
                    previous_nodes[next_stop] = current_node
                    heapq.heappush(pq, (new_distance, next_stop))

    path = []
    bus_trip = y
    path.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    while bus_trip != x:
        bus_trip = previous_nodes.get(bus_trip)
        if bus_trip is None:
            return []  
        path.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    return path[::-1] # skila réttri röð

meistaravellir_to_fb = find_path_new(graph, walk_graph, '90000097', '90000446', "11:30:00")
print(meistaravellir_to_fb)


In [ ]:
import heapq

def distance_in_kilometers(x, y): # 800 m á mínútu
    return max( calculate_distance(stop_locations[x], stop_locations[y]) / 800 - 0.1, 0)

def find_path_new_astar(g, g_walk, x, y, t):
    t = time_to_minutes(t)
    if x not in g:
        raise Exception(x + " er ekki í netinu")

    pq = []
    heapq.heappush(pq, (t + distance_in_kilometers(x,y), x))

    earliest = {node: float('inf') for node in g}
    earliest[x] = t + distance_in_kilometers(x,y)

    real_time = {node: float('inf') for node in g}
    real_time[x] = t

    previous_nodes = {}


    while pq:
        current_distance, current_node = heapq.heappop(pq)

        if current_node == y:
            break

        if current_distance > earliest[current_node]:
            continue

        
        for edge in get_edges(g, current_node):
            next_stop, travel_time = edge[1], time_to_minutes(edge[0])
            new_distance = travel_time
            if new_distance < real_time[next_stop] and travel_time >= real_time[current_node]:
                earliest[next_stop] = new_distance + distance_in_kilometers(next_stop,y)
                real_time[next_stop] = new_distance
                previous_nodes[next_stop] = current_node
                heapq.heappush(pq, (new_distance + distance_in_kilometers(next_stop,y), next_stop))

        if current_node in g_walk:
            for edge in get_edges(g_walk, current_node):
                next_stop, walking_time = edge[1], edge[0] / 40  
                new_distance = current_distance + walking_time
                if new_distance < real_time[next_stop]:
                    earliest[next_stop] = new_distance + distance_in_kilometers(next_stop,y)
                    real_time[next_stop] = new_distance
                    previous_nodes[next_stop] = current_node
                    heapq.heappush(pq, (new_distance + distance_in_kilometers(next_stop,y), next_stop))

    path = []
    bus_trip = y
    path.append(vertex_info[bus_trip] + [minutes_to_time(real_time[bus_trip])])
    while bus_trip != x:
        bus_trip = previous_nodes.get(bus_trip)
        if bus_trip is None:
            return []  
        path.append(vertex_info[bus_trip] + [minutes_to_time(real_time[bus_trip])])
    return path[::-1] # skila réttri röð

meistaravellir_to_fb = find_path_new_astar(graph, walk_graph, '90000097', '90000446', "11:30:00")
print(meistaravellir_to_fb)


In [ ]:
import time

def time_find_path_new(graph, walk_graph, test_cases):
    elapsed_times = [] 
    for x, y, t in test_cases:
        start_time = time.time()
        find_path_new(graph, walk_graph, x, y, t)
        end_time = time.time()
        elapsed = end_time - start_time 
        elapsed_times.append(elapsed)
    
    average_time = sum(elapsed_times) / len(elapsed_times)
    requests_per_second = 1 / average_time if average_time > 0 else float('inf')
    print(f"Average execution time: {average_time:.4f} seconds")
    print(f"Requests per second: {requests_per_second:.2f}")


test_cases = [
    ('90000097', '90000446', "11:30:00"), # meistaravellir to fb
    ('10000802', '10000804', "12:30:00"), # hamraborg to sunnuhlíð
    ('90000410', '90000296', "08:00:00"), # skógarás to háskóli íslands
    ('90000780', '10000875', "15:50:00"), # borgartún to smáralind
    ('14001641', '90000173', "17:30:00"), # flatahraun to dunhagi
    ('13001337', '13001300', "18:09:00"), # Asparlundur to Ásgarður
    ('16040620', '90000213', "20:30:00"), # Fmos to laugardalslaug
    ('90000351', '90000461', "11:43:00"), # ártún to kringlan
    ('90000077', '90000301', "09:15:00"), # bústaðakirkja to perlan
    ('90000446', '90000097', "11:30:00"), # fb to meistaravellir
    ]

time_find_path_new(graph, walk_graph, test_cases)


In [136]:
# Færri skiptingar
import heapq

def add_vertex_new(g, v):
    if v in g:
        raise Exception("Vertex already in graph")
    else:
        g[v] = []
    return g

def add_edge_new(g, fro, to, data, route):
    if fro not in g or to not in g:
        raise Exception(f"{fro} or {to} not in the network")
    g[fro].append({'travel_time': time_to_minutes(data), 'destination': to, 'route': route})


def get_edges_new(g, v):
    edges = g.get(v, [])
    return edges

# Making the new graph for less changes
graph_fewer_changes = {}
vertex_info = {}

for x in stoppistodvar:
    graph_fewer_changes = add_vertex_new(graph_fewer_changes, x[0])
    vertex_info[x[0]] = [x[0], stoppistodvar_id_to_name[x[0]], 'stöð']

for trip_id, stops in paths.items():
    route = trip_to_straeto[trip_id]
    for s in range(len(stops)-1):
        interval_name = 'p' + trip_id + '_' + stops[s][4]
        add_vertex_new(graph_fewer_changes, interval_name)
        vertex_info[interval_name] = [trip_id, route, 'strætó']
        add_edge_new(graph_fewer_changes, stops[s][3], interval_name, stops[s][2], route)
        add_edge_new(graph_fewer_changes, interval_name, stops[s+1][3], stops[s+1][1], route)


# Adding walking edges to the new graph
MAX_WALKING_DISTANCE = 300  

def add_walking_edges_new(graph, stop_locations):
    stop_ids = list(stop_locations.keys())
    for i, stop_id1 in enumerate(stop_ids):
        for stop_id2 in stop_ids[i+1:]:  
            distance = calculate_distance(stop_locations[stop_id1], stop_locations[stop_id2])
            if distance <= MAX_WALKING_DISTANCE:
                try:
                    graph = add_edge_new(graph, stop_id1, stop_id2, distance, None)
                    graph = add_edge_new(graph, stop_id2, stop_id1, distance, None)
                except Exception as e:
                    print('Exception', e) 
    return graph


# Making the new graph with walking edges
walk_graph_new = {}

for x in stoppistodvar:
  walk_graph_new = add_vertex_new(walk_graph_new, x[0])
walk_graph_new = add_walking_edges_new(walk_graph_new, stop_locations)

# print(len(walk_graph_new))

# for x in walk_graph_new['20000254']:
#   print(x)

# Finding the path with less changes
def find_path_new_less_changing(g, g_walk, x, y, start_time):
    if x not in g or y not in g:
        raise ValueError("Start or end node not found in the graph")

    start_time = time_to_minutes(start_time)
    pq = []
    heapq.heappush(pq, (0, start_time, x, None)) 
    earliest = {node: float('inf') for node in g}
    earliest[x] = start_time
    previous_nodes = {}

    while pq:
        route_changes, current_time, current_node, last_route = heapq.heappop(pq)

        if current_node == y:
            break
        if current_time > earliest[current_node]:
            continue

        for edge in get_edges_new(g, current_node):
            travel_time = edge['travel_time']
            next_stop = edge['destination']
            route = edge['route']
            new_time = current_time + travel_time
            change_cost = route_changes + (1 if route != last_route and last_route is not None else 0)

            if new_time < earliest[next_stop] or (new_time == earliest[next_stop] and change_cost < route_changes):
                earliest[next_stop] = new_time
                previous_nodes[next_stop] = (current_node, route)
                heapq.heappush(pq, (change_cost, new_time, next_stop, route))

        if current_node in g_walk:
            for edge in get_edges_new(g_walk, current_node):
                walking_time = edge['travel_time']
                next_stop = edge['destination']
                new_time = current_time + walking_time

                if new_time < earliest[next_stop]:
                    earliest[next_stop] = new_time
                    previous_nodes[next_stop] = (current_node, last_route)
                    heapq.heappush(pq, (route_changes, new_time, next_stop, last_route))

    if y not in previous_nodes:
        return "No path found"

    path = []
    node = y
    while node != x:
        if node not in previous_nodes:
            return "Incomplete path: could not trace back to start"
        prev_node, route = previous_nodes[node]
        path.append(vertex_info[node] + [route, minutes_to_time(earliest[node])])
        node = prev_node

    return path[::-1]


# Test cases
# find_path_new_less_changing(graph_fewer_changes, walk_graph_new, '90000018', '90000405', "11:30:00")
# find_path_new_less_changing(graph_fewer_changes, walk_graph_new, '90000436', '90000287', "11:30:00")

# ártún A til kringlumýrarbraut
find_path_new_less_changing(graph_fewer_changes, walk_graph_new, '90000019', '90000016', "20:30:00")



Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'
Exception 'float' object has no attribute 'split'


[['526190', 'ST.6', 'strætó', 'ST.6', '27:06'],
 ['90000414', 'Sogamýri', 'stöð', 'ST.6', '33:43'],
 ['526190', 'ST.6', 'strætó', 'ST.6', '40:20'],
 ['90000415', 'Skeifan', 'stöð', 'ST.6', '46:58'],
 ['526190', 'ST.6', 'strætó', 'ST.6', '53:36'],
 ['90000013', 'Kringlan', 'stöð', 'ST.6', '60:16'],
 ['525486', 'ST.4', 'strætó', 'ST.4', '66:59'],
 ['90000015', 'Kringlumýrarbraut', 'stöð', 'ST.4', '73:44'],
 ['525486', 'ST.4', 'strætó', 'ST.4', '80:29'],
 ['10010802', 'Hamraborg A', 'stöð', 'ST.4', '87:17'],
 ['525579', 'ST.4', 'strætó', 'ST.4', '93:53'],
 ['10000859', 'Kópavogsskóli', 'stöð', 'ST.4', '100:29'],
 ['525579', 'ST.4', 'strætó', 'ST.4', '107:05'],
 ['10000820', 'MK', 'stöð', 'ST.4', '113:42'],
 ['525579', 'ST.4', 'strætó', 'ST.4', '120:19'],
 ['10000857', 'Brattabrekka', 'stöð', 'ST.4', '126:57'],
 ['525579', 'ST.4', 'strætó', 'ST.4', '133:35'],
 ['10000819', 'Víghóll', 'stöð', 'ST.4', '140:14'],
 ['525579', 'ST.4', 'strætó', 'ST.4', '146:53'],
 ['10000846', 'Lundarbrekka', '